In [1]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util, models
from sklearn.feature_extraction import _stop_words as stop_words
from tqdm.notebook import tqdm
from rank_bm25 import BM25Okapi

import torch
import string
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahmedalsayed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
bi_enc_weights = '../models/bienc-exp7/'
cr_enc_weights = '../models/crenc-readme-exp4/'
data_folder = 'generated5'
top_k = 50
use_base = False

In [3]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):
    # Basic cleaning (e.g., removing special characters, non-alphanumeric characters)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Reconstruct the text
    return ' '.join(tokens)
file_path = '../data/20231004_data.xlsx' 
df = pd.read_excel(file_path, index_col=0)

df['readme_short'] = df['readme_short'].astype(str)
df['docker'] = df['docker'].astype(str)
df['answer'] = df.apply(lambda row: advanced_clean(' '.join(row['readme_short'].split()[:400] + row['docker'].split()[:200])), axis=1)


df.head(2)





[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ahmedalsayed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahmedalsayed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ahmedalsayed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Question Link,Question Title,Question Body,Accepted Answer Body,link,readme,docker,readme_short,answer
0,390150,Authenticating against Active Directory with J...,<p>I have a simple task of authenticating agai...,<p>Here's the code I put together based on exa...,https://github.com/jenkinsci/active-directory-...,Active Directory plugin for Jenkins\n=========...,Dockerfile from src/test/resources/fixture/Doc...,Active Directory plugin for Jenkins ==========...,active directory plugin jenkins build status j...
1,1197678,Using Thrift with Delphi Win32,<p>I'm interested in connecting to the Evernot...,<p><strong>Old Answer Replaced thanks to Leo:<...,https://github.com/apache/thrift,Apache Thrift\n=============\nIntroduction\n==...,Dockerfile from build/docker/msvc2017/Dockerfi...,Apache Thrift ============= Introduction =====...,apache thrift introduction thrift lightweight ...


In [4]:
english_stopwords = set(stopwords.words('english'))

def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in english_stopwords:
      tokenized_doc.append(token)
      
  return tokenized_doc

In [5]:
if use_base:
    word_embedding_model = models.Transformer('distilroberta-base', max_seq_length=350)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    cr_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')
else:
    bi_encoder = SentenceTransformer(bi_enc_weights)
    cr_encoder = CrossEncoder(cr_enc_weights)

In [7]:
with open('/Users/Documents/final-code-microservice-paper/experiments/test_passage_100.json', 'r') as f:
    val_passage = json.load(f)

with open('/Users/Documents/final-code-microservice-paper/experiments/test_corpus_100.json', 'r') as f:  # Note the _100 in the filename
    val_corpus = json.load(f)

val_query_answer = {}
val_query_readme = {}

for idx, rel in val_passage.items():
    if idx not in val_corpus:  
        continue  

    pos = rel[0]  

   
    try:
        readme = df.loc[pos, 'answer']
    except (KeyError, IndexError):
       
        continue

    questions = []
    for p in rel:  
       
        try:
            question = df.loc[p, 'Question Title']
            questions.append(question)
        except (KeyError, IndexError):
            
            pass

   
    if questions:
        val_query_answer[idx] = questions
        val_query_readme[idx] = [readme]





In [8]:
def shorten(text):
    tmp = text.split()[:512]
    return ' '.join(tmp)
 

val_text = list(val_corpus.values())

val_readme = []
for t in val_text:
    val_readme.append(df.loc[df['Question Title'] == t, 'answer'].values[0])


with open("/Users/Documents/final-code-microservice-paper/experiments/data/embeddings_GPT.json", "r") as jsonfile:
    embeddings_dict = json.load(jsonfile)


val_emb_tensors = []



title_to_id = dict(zip(df['Question Title'], df.index))


for text in val_text:
    
    text_id = title_to_id[text]
    embedding = embeddings_dict.get(str(text_id))  
    if embedding:  
        val_emb_tensors.append(torch.tensor(embedding))


val_emb = torch.stack(val_emb_tensors)


In [9]:
from tqdm import tqdm
tokenized_corpus = []
for idx, passage in tqdm(val_corpus.items()):
    tokenized_corpus.append(bm25_tokenizer(passage))

100%|██████████| 100/100 [00:00<00:00, 248625.01it/s]


In [10]:
#Total Questions Answered Correctly
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score, ndcg_score
import warnings
warnings.filterwarnings("ignore", message="No positive class found in y_true, recall is set to one for all thresholds.")

def compute_cosine_similarity(query_embedding, corpus_embeddings):
    # Normalize embeddings
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding / torch.norm(query_embedding, keepdim=True)
        query_embedding = query_embedding.unsqueeze(0)  # Add an additional dimension
    else:
        query_embedding = query_embedding / torch.norm(query_embedding, dim=1, keepdim=True)
    
    corpus_embeddings = corpus_embeddings / torch.norm(corpus_embeddings, dim=1, keepdim=True)
    
    # Compute cosine similarity
    cosine_similarities = torch.mm(query_embedding, corpus_embeddings.transpose(0, 1))
    
    return cosine_similarities




def forward_pass_rerank(query, precomputed_embedding=None, val_embeddings=None, top_k=50):
    
    if precomputed_embedding is None:
        q_emb = bi_encoder.encode(query, convert_to_tensor=True)
    else:
        q_emb = precomputed_embedding

    # Ensure the query embedding is 2-dimensional
    if len(q_emb.shape) == 1:
        q_emb = q_emb.unsqueeze(0)

    if val_embeddings is None:
        raise ValueError("No embeddings provided for the validation set.")
    
    
    if val_embeddings.shape[1] != q_emb.shape[1]:
        val_embeddings = val_embeddings.transpose(0, 1)
    
    cosine_similarities = compute_cosine_similarity(q_emb, val_embeddings)
    

    top_indices = torch.topk(cosine_similarities, k=top_k+1, dim=1).indices[0].tolist()
    hits = [{'corpus_id': index, 'score': cosine_similarities[0, index].item()} for index in top_indices]
   

    cross_inputs = []
    to_remove = -1
    for hit in hits:
        readme = val_readme[hit['corpus_id']]
        text = val_text[hit['corpus_id']]
        if query == text:
            to_remove = hits.index(hit)
        cross_inputs.append([query, readme])
    cross_scores = cr_encoder.predict(cross_inputs)
    for idx in range(len(cross_scores)):
        hits[idx]['cross_score'] = cross_scores[idx]
    if to_remove != -1: del hits[to_remove]
    hits = hits[:top_k]

    return hits


total_questions_answered_correctly = 0

for (query_key, answers), (_, readme) in tqdm(zip(val_query_answer.items(), val_query_readme.items()), total=len(val_query_answer)):
    query= val_corpus[query_key]
    # Fetch precomputed embedding
    query_id = title_to_id[query]
    precomputed_query_embedding = torch.tensor(embeddings_dict.get(str(query_id)))
    hits = forward_pass_rerank(query, precomputed_embedding=precomputed_query_embedding, val_embeddings=val_emb, top_k=top_k)

    r = val_query_readme[query_key]
    
    hit_info_list = []
    
    for hit in hits[:5]:
        readme = val_readme[hit['corpus_id']]
        cscore_output_cr = cr_encoder.predict([query, readme])  
        cscore_cr = cscore_output_cr[0] if isinstance(cscore_output_cr, (list, tuple, np.ndarray)) else cscore_output_cr
        hit_info = {
            'h_text': readme,
            'cscore': cscore_cr,
            'is_correct': "Yes" if readme in r else "No"
        }
        hit_info_list.append(hit_info)
    
    question_answered_correctly = any(hit_info['is_correct'] == "Yes" for hit_info in hit_info_list)
    if question_answered_correctly:
        total_questions_answered_correctly += 1

print(f"Total Questions Answered Correctly: {total_questions_answered_correctly}")

100%|██████████| 100/100 [07:43<00:00,  4.64s/it]

Total Questions Answered Correctly: 63


In [11]:
#Total Correcte Answeres 
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score, ndcg_score
import warnings
warnings.filterwarnings("ignore", message="No positive class found in y_true, recall is set to one for all thresholds.")

def compute_cosine_similarity(query_embedding, corpus_embeddings):
    # Normalize embeddings
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding / torch.norm(query_embedding, keepdim=True)
        query_embedding = query_embedding.unsqueeze(0)  # Add an additional dimension
    else:
        query_embedding = query_embedding / torch.norm(query_embedding, dim=1, keepdim=True)
    
    corpus_embeddings = corpus_embeddings / torch.norm(corpus_embeddings, dim=1, keepdim=True)
    
    # Compute cosine similarity
    cosine_similarities = torch.mm(query_embedding, corpus_embeddings.transpose(0, 1))
    
    return cosine_similarities



def forward_pass_rerank(query, precomputed_embedding=None, val_embeddings=None, top_k=50):
    
    if precomputed_embedding is None:
        q_emb = bi_encoder.encode(query, convert_to_tensor=True)
    else:
        q_emb = precomputed_embedding

    # Ensure the query embedding is 2-dimensional
    if len(q_emb.shape) == 1:
        q_emb = q_emb.unsqueeze(0)

    if val_embeddings is None:
        raise ValueError("No embeddings provided for the validation set.")
    
    
    if val_embeddings.shape[1] != q_emb.shape[1]:
        val_embeddings = val_embeddings.transpose(0, 1)
    
    cosine_similarities = compute_cosine_similarity(q_emb, val_embeddings)
    

    top_indices = torch.topk(cosine_similarities, k=top_k+1, dim=1).indices[0].tolist()
    hits = [{'corpus_id': index, 'score': cosine_similarities[0, index].item()} for index in top_indices]
   

    cross_inputs = []
    to_remove = -1
    for hit in hits:
        readme = val_readme[hit['corpus_id']]
        text = val_text[hit['corpus_id']]
        if query == text:
            to_remove = hits.index(hit)
        cross_inputs.append([query, readme])
    cross_scores = cr_encoder.predict(cross_inputs)
    for idx in range(len(cross_scores)):
        hits[idx]['cross_score'] = cross_scores[idx]
    if to_remove != -1: del hits[to_remove]
    hits = hits[:top_k]

    return hits

total_correct = 0

for (query_key, answers), (_, readme) in tqdm(zip(val_query_answer.items(), val_query_readme.items()), total=len(val_query_answer)):
    query= val_corpus[query_key]
    # Fetch precomputed embedding
    query_id = title_to_id[query]
    precomputed_query_embedding = torch.tensor(embeddings_dict.get(str(query_id)))
    hits = forward_pass_rerank(query, precomputed_embedding=precomputed_query_embedding, val_embeddings=val_emb, top_k=top_k)

    r = val_query_readme[query_key]
    
    hit_info_list = []
    
    for hit in hits[:5]:
        readme = val_readme[hit['corpus_id']]
        cscore_output_cr = cr_encoder.predict([query, readme])  
        cscore_cr = cscore_output_cr[0] if isinstance(cscore_output_cr, (list, tuple, np.ndarray)) else cscore_output_cr
        hit_info = {
            'h_text': readme,
            'cscore': cscore_cr,
            'is_correct': "Yes" if readme in r else "No"
        }
        hit_info_list.append(hit_info)
    
    correct_count = sum(1 for hit_info in hit_info_list if hit_info['is_correct'] == "Yes")
    total_correct += correct_count

print(f"Total Correct Answers: {total_correct}")

100%|██████████| 100/100 [07:42<00:00,  4.63s/it]

Total Correct Answers: 185


In [13]:
# print the question where more than 2 hits are correct :

import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score, ndcg_score
import warnings
warnings.filterwarnings("ignore", message="No positive class found in y_true, recall is set to one for all thresholds.")

def compute_cosine_similarity(query_embedding, corpus_embeddings):
    # Normalize embeddings
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding / torch.norm(query_embedding, keepdim=True)
        query_embedding = query_embedding.unsqueeze(0)  # Add an additional dimension
    else:
        query_embedding = query_embedding / torch.norm(query_embedding, dim=1, keepdim=True)
    
    corpus_embeddings = corpus_embeddings / torch.norm(corpus_embeddings, dim=1, keepdim=True)
    
    # Compute cosine similarity
    cosine_similarities = torch.mm(query_embedding, corpus_embeddings.transpose(0, 1))
    
    return cosine_similarities



def forward_pass_rerank(query, precomputed_embedding=None, val_embeddings=None, top_k=50):
    
    if precomputed_embedding is None:
        q_emb = bi_encoder.encode(query, convert_to_tensor=True)
    else:
        q_emb = precomputed_embedding

    # Ensure the query embedding is 2-dimensional
    if len(q_emb.shape) == 1:
        q_emb = q_emb.unsqueeze(0)

    if val_embeddings is None:
        raise ValueError("No embeddings provided for the validation set.")
    
    
    if val_embeddings.shape[1] != q_emb.shape[1]:
        val_embeddings = val_embeddings.transpose(0, 1)
    
    cosine_similarities = compute_cosine_similarity(q_emb, val_embeddings)
    

    top_indices = torch.topk(cosine_similarities, k=top_k+1, dim=1).indices[0].tolist()
    hits = [{'corpus_id': index, 'score': cosine_similarities[0, index].item()} for index in top_indices]
   

    cross_inputs = []
    to_remove = -1
    for hit in hits:
        readme = val_readme[hit['corpus_id']]
        text = val_text[hit['corpus_id']]
        if query == text:
            to_remove = hits.index(hit)
        cross_inputs.append([query, readme])
    cross_scores = cr_encoder.predict(cross_inputs)
    for idx in range(len(cross_scores)):
        hits[idx]['cross_score'] = cross_scores[idx]
    if to_remove != -1: del hits[to_remove]
    hits = hits[:top_k]

    return hits



total_correct = 0

for (query_key, answers), (_, readme) in tqdm(zip(val_query_answer.items(), val_query_readme.items()), total=len(val_query_answer)):
    query= val_corpus[query_key]
    # Fetch precomputed embedding
    query_id = title_to_id[query]
    precomputed_query_embedding = torch.tensor(embeddings_dict.get(str(query_id)))
    hits = forward_pass_rerank(query, precomputed_embedding=precomputed_query_embedding, val_embeddings=val_emb, top_k=top_k)

    r = val_query_readme[query_key]
    
    hit_info_list = []

    for hit in hits[:5]:
        readme = val_readme[hit['corpus_id']]
        h_text = val_text[hit['corpus_id']]
        cscore_output_cr = cr_encoder.predict([query, readme])  
        cscore_cr = cscore_output_cr[0] if isinstance(cscore_output_cr, (list, tuple, np.ndarray)) else cscore_output_cr
        hit_info = {
            'h_text': h_text,
            'cscore': cscore_cr,
            'is_correct': "Yes" if readme in r else "No"
        }
        hit_info_list.append(hit_info)
    
    correct_count = sum(1 for hit_info in hit_info_list if hit_info['is_correct'] == "Yes")
    
    # Check if more than 2 hits are correct, and if so, print the question and its hits
    if correct_count >= 2:
        print(f"Question: {query}")
        for hit_info in hit_info_list:
            print(f"\tHit: {hit_info['h_text']} - Correct: {hit_info['is_correct']}")
        print("\n")
    
    total_correct += correct_count

print(f"Total Correct Answers: {total_correct}")


  1%|          | 1/100 [00:04<07:52,  4.77s/it]

Question: Start new K8s Pod within K8s Pod without "Privileges"
	Hit: Start kubernetes container with specific command - Correct: No
	Hit: Kubernetes simple authentication - Correct: Yes
	Hit: Kubernetes Storage on bare-metal/private cloud - Correct: Yes
	Hit: Kubernetes kube-dns pod is pending - Correct: Yes
	Hit: kubernetes get values from already deployed pod/daemonset - Correct: Yes




  2%|▏         | 2/100 [00:09<07:38,  4.67s/it]

Question: Service proxy with affinity based on URL
	Hit: Access nodeport via kube-proxy from another machine - Correct: Yes
	Hit: Dynamically resolve a pods address for get request - Correct: Yes
	Hit: Mapping an external http load balancer to a Kubernetes Cluster - Correct: No
	Hit: AWS EKS Guestbook url not accessible - Correct: No
	Hit: Bind to multiple ip addresses in a single docker container - Correct: No




  3%|▎         | 3/100 [00:14<07:31,  4.65s/it]

...

Question: Get current resource usage of a pod in Kubernetes with Go client
	Hit: Kubernetes Watch Pod Events with api - Correct: Yes
	Hit: kubernetes get values from already deployed pod/daemonset - Correct: Yes
	Hit: Kubernetes prometheus metrics for running pods and nodes? - Correct: Yes
	Hit: Can I get a Pod's pause container ID via the Kubernetes API? - Correct: Yes
	Hit: Dynamically resolve a pods address for get request - Correct: Yes




 99%|█████████▉| 99/100 [07:43<00:04,  4.70s/it]

Question: kubelet fails to start with rocket
	Hit: kubeadm init fails with incorrect docker version - Correct: Yes
	Hit: kubernetes is not routing traffic to grpc service - Correct: No
	Hit: Kubernetes kube-dns pod is pending - Correct: Yes
	Hit: Kubernetes with Vagrant nodes won't register - Correct: Yes
	Hit: Airflow scheduler can not connect to Kubernetes service api - Correct: No




100%|██████████| 100/100 [07:48<00:00,  4.69s/it]

Question: Where is the latest protobuf API for the Kubernetes CRI?
	Hit: Kubernetes prometheus metrics for running pods and nodes? - Correct: Yes
	Hit: Kubernetes Watch Pod Events with api - Correct: Yes
	Hit: kubernetes is not routing traffic to grpc service - Correct: No
	Hit: What is the syntax of annotation values in Kubernetes? - Correct: Yes
	Hit: Restricted Kubernetes dashboard? - Correct: No


Total Correct Answers: 185
